In [1]:
!pip install duckdb==0.5.1
!pip install pytorch_tabular[extra]
!pip install torch
!pip install lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 55.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.7 MB/s eta 0:00:0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.0/546.0 KB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 6.5 MB/s eta 0:00:00
 

In [1]:
import sys,os
project_path = "/content/drive/MyDrive/Work/quantitative_research/"
#database_path = "/content/drive/MyDrive/Work/trading_updated/database/development/develop"

sys.path.append(project_path)
#from featurewiz import featurewiz
import random
import lightgbm as lgb
import numpy as np
import sklearn.metrics as mt
import duckdb
import pandas as pd
import data.utils.duckdb_utils as ddu
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
import numpy as np
import pandas as pd
import os
import lightning.pytorch as pl


In [2]:
database_path = "/content/drive/Othercomputers/My Mac/google_drive/database/production/banknifty_final1.duckdb"
db_connection = duckdb.connect(database=database_path , read_only=False)
train_table = 'banknifty.train'
val_table = 'banknifty.validation'
buffer_table = 'banknifty.buffer'

In [3]:
data_suffix = 'improved_one'
train_feature_table = f'banknifty.train_feature_{data_suffix}'
train_feature_info_table = f'banknifty.train_feature_info_{data_suffix}'
train_feature_selection_table = f'banknifty.train_feature_selection_{data_suffix}'
train_training_info_table = f'banknifty.train_training_info_{data_suffix}'
train_tuning_info_table = f'banknifty.train_tuning_info_{data_suffix}'
ohlcraw_table = 'banknifty.ohlc_raw_data'

train_feature_selection_table_df = ddu.load_table_df(db_connection,table_name=train_feature_selection_table,
                                             column_names=None,filter=None,load_sql=None)



In [4]:
target = 'label__minus_15_1min_close__label_generator_4class'
cols = train_feature_selection_table_df[train_feature_selection_table_df['label_name']==target]['feature_name'].tolist()

In [5]:
len(cols)

366

In [6]:
sql = f"Select count(*) as cnt, time_split from {train_table} group by time_split"
a = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
train_cols = [col for col in a['time_split'].tolist() if 'train' in str(col)]

sql = f"Select count(*) as cnt, time_split from {val_table} group by time_split"
a = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
val_cols = [col for col in a['time_split'].tolist() if 'validation' in str(col)]

sql = f"Select count(*) as cnt, time_split from {buffer_table} group by time_split"
a = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
buffer_cols = [col for col in a['time_split'].tolist() if 'buffer' in str(col)]


In [7]:
len(train_cols)

10

In [8]:
from fastai.tabular.core import df_shrink_dtypes,df_shrink,add_datepart

In [9]:
cat_cols = ['timestampYear',
 'timestampMonth',
 'timestampWeek',
 'timestampDay',
 'timestampDayofweek',
 'timestampDayofyear',
 'timestampIs_month_end',
 'timestampIs_month_start',
 'timestampIs_quarter_end',
 'timestampIs_quarter_start',
 'timestampIs_year_end',
 'timestampIs_year_start',
 'timestampElapsed']

In [27]:
sql = f'''select {",".join(cols)},{target},timestamp from {train_table} where time_split in 
('{"','".join(random.sample(train_cols, 10)) }') and
{target} != 'unknown' 
'''

train = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
train = add_datepart (train, field_name ='timestamp', prefix=None, drop=True, time=False)

train= df_shrink(train)
train = train.dropna()
train[target] = train[target].map({'neutral':1,'call':2,'put':3})

for c in cat_cols:
  train[c] = train[c].astype(str)
  #train[c] = train[c].astype('category')

In [28]:
train.shape

(471766, 380)

In [29]:
sql = f'''select {",".join(cols)},{target},timestamp  from {val_table} where time_split in ('{"','".join(random.sample(val_cols, 5)) }') and
{target} != 'unknown' 
'''
print(sql)
valid = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
valid = add_datepart (valid, field_name ='timestamp', prefix=None, drop=True, time=False)

valid= df_shrink(valid)
valid = valid.dropna()
valid[target] = valid[target].map({'neutral':1,'call':2,'put':3})
for c in cat_cols:
  valid[c] = valid[c].astype(str)
  #valid[c] = valid[c].astype('category')
valid.shape

select ROLLSTT_120_2_MEANDIFF_close,ROLLSTT_45_2_MAXMIN_close20,ROLLRNK_EMA_15_ema_close_15_None,ROLLDIFF_0_0_super_diff13,ROLLRNK_EMA_45_ema_close10_60_None,ROLLSTT_120_2_MAXMIN_close,TRUERANGE_120_hlc,ROLLSTT_30_2_MAXMIN_close,ROLLSTT_15_2_MAXMIN_close,ROLLPCM_DCL_60_60_don4_30_None,ROLLRNK_EMA_45_ema_close10_15_None,ROLLPCM_DCU_30_30_don2_30_None,ROLLRNK_EMA_120_ema_close20_15_None,DIFFCOL_DCU_20_20_hl50_close,ROLLPCM_DCU_60_60_don4_60_None,ROLLSTT_60_2_MAXMIN_close20,ROLLPCM_SUPERTs_45_30_supertrend_close_30_None,ROLLPCM_DCL_30_30_don2_30_None,ROLLPCM_DCU_240_240_don5_15_None,DIFFCOL_DCL_60_60_don4_close,ADX_30_adx14,ROLLBIN_100_0_True_5_close,ROLLPCM_DCL_120_120_don9_60_None,ROLLPCM_SUPERTs_15_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_60_None,ROLLPCM_DCM_30_30_don7_30_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_15_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_60_None,ROLLPCM_DCM_120_120_don14_15_None,ROLLPCM_DCL_240_240_don5_60_None,RO

(89465, 380)

In [30]:
sql = f'''select {",".join(cols)},{target},timestamp from {buffer_table} where time_split in ('{"','".join(random.sample(buffer_cols, 5)) }') and
{target} != 'unknown' 
'''
print(sql)
test = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
test = add_datepart (test, field_name ='timestamp', prefix=None, drop=True, time=False)

test= df_shrink(test)
test = test.dropna()
test[target] = test[target].map({'neutral':1,'call':2,'put':3})

for c in cat_cols:
  test[c] = test[c].astype(str)
  #test[c] = test[c].astype('category')
test.shape

select ROLLSTT_120_2_MEANDIFF_close,ROLLSTT_45_2_MAXMIN_close20,ROLLRNK_EMA_15_ema_close_15_None,ROLLDIFF_0_0_super_diff13,ROLLRNK_EMA_45_ema_close10_60_None,ROLLSTT_120_2_MAXMIN_close,TRUERANGE_120_hlc,ROLLSTT_30_2_MAXMIN_close,ROLLSTT_15_2_MAXMIN_close,ROLLPCM_DCL_60_60_don4_30_None,ROLLRNK_EMA_45_ema_close10_15_None,ROLLPCM_DCU_30_30_don2_30_None,ROLLRNK_EMA_120_ema_close20_15_None,DIFFCOL_DCU_20_20_hl50_close,ROLLPCM_DCU_60_60_don4_60_None,ROLLSTT_60_2_MAXMIN_close20,ROLLPCM_SUPERTs_45_30_supertrend_close_30_None,ROLLPCM_DCL_30_30_don2_30_None,ROLLPCM_DCU_240_240_don5_15_None,DIFFCOL_DCL_60_60_don4_close,ADX_30_adx14,ROLLBIN_100_0_True_5_close,ROLLPCM_DCL_120_120_don9_60_None,ROLLPCM_SUPERTs_15_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_60_None,ROLLPCM_DCM_30_30_don7_30_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_15_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_60_None,ROLLPCM_DCM_120_120_don14_15_None,ROLLPCM_DCL_240_240_don5_60_None,RO

(73545, 380)

In [8]:
# store the category code mapping, so we can encode any new incoming data
# other than our training set
cat_code_dict = {}
for col in cat_cols:
    category_col = train[col].astype('category')
    cat_code_dict[col] = {value: idx for idx, value in enumerate(category_col.cat.categories)} 

cat_code_dict

NameError: ignored

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train[cols])

In [33]:
def preprocess(df, scaler, cat_code_dict, num_cols, cat_cols, label_col):
    df = df.copy()

    # numeric fields
    df[num_cols] = scaler.transform(df[num_cols])
    df[num_cols] = df[num_cols].astype(np.float32)

    # categorical fields
    for col in cat_cols:
        code_dict = cat_code_dict[col]
        code_fillna_value = len(code_dict)
        df[col] = df[col].map(code_dict).fillna(code_fillna_value).astype(np.int64)

    # label
    df[label_col] = df[label_col].astype(np.float32)
    return df

In [34]:
df_groups = {
    'train': train,
    'val': valid,
    'test': test
}

data_dir = 'onnx_data'
os.makedirs(data_dir, exist_ok=True)

for name, df_group in df_groups.items():
    filename = os.path.join(data_dir, f'{name}.csv')
    df_preprocessed = preprocess(df_group, scaler, cat_code_dict, cols, cat_cols, target)
    df_preprocessed.to_csv(filename, index=False)

df_preprocessed.dtypes

ROLLSTT_120_2_MEANDIFF_close          float32
ROLLSTT_45_2_MAXMIN_close20           float32
ROLLRNK_EMA_15_ema_close_15_None      float32
ROLLDIFF_0_0_super_diff13             float32
ROLLRNK_EMA_45_ema_close10_60_None    float32
                                       ...   
timestampIs_quarter_end                 int64
timestampIs_quarter_start               int64
timestampIs_year_end                    int64
timestampIs_year_start                  int64
timestampElapsed                        int64
Length: 380, dtype: object

In [15]:
target

'label__minus_15_1min_close__label_generator_4class'

In [35]:
import gc
gc.enable()
del train,valid,test,df_groups,df_preprocessed
gc.collect()

0

### start 

In [10]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [11]:
class TabularDataset(Dataset):

    def __init__(self, path, num_cols, cat_cols, label_col):
        self.path = path
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.label_col = label_col
        self.df = read_data(path, num_cols, cat_cols, label_col)
        self.df= df_shrink(self.df)
        print(self.df[label_col].value_counts())

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        num_array = self.df[self.num_cols].iloc[idx].values
        cat_array = self.df[self.cat_cols].iloc[idx].values
        label_array = self.df[self.label_col].iloc[idx]
        return num_array, cat_array, label_array


def read_data(path, num_cols, cat_cols, label_col):
    float_cols = num_cols + [label_col]
    dtype = {col: np.float32 for col in float_cols}
    dtype.update({col: np.int64 for col in cat_cols})
    return pd.read_csv(path, dtype=dtype)

In [12]:
batch_size = 3
data_dir = 'onnx_data'
path_train = os.path.join(data_dir, 'train.csv')
x1 = pd.read_csv(path_train)
cat_code_dict = {}
for col in cat_cols:
    category_col = x1[col].astype('category')
    cat_code_dict[col] = {value: idx for idx, value in enumerate(category_col.cat.categories)} 
del x1
cat_code_dict



{'timestampYear': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12},
 'timestampMonth': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11},
 'timestampWeek': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51},
 'timestampDay': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,

In [13]:
dataset = TabularDataset(path_train, cols, cat_cols, target)
data_loader = DataLoader(dataset, batch_size)

1.0    328879
3.0     71510
2.0     71377
Name: label__minus_15_1min_close__label_generator_4class, dtype: int64


In [14]:
del dataset,data_loader

In [16]:


# our data loader now returns batches of numerical/categorical/label tensor
num_tensor, cat_tensor, label_tensor = next(iter(data_loader))

#print('numerical value tensor:\n', num_tensor)
#print('categorical value tensor:\n', cat_tensor)
print('label tensor:\n', label_tensor)

label tensor:
 tensor([2., 2., 2.])


In [15]:
class TabularDataModule(pl.LightningDataModule):

    def __init__(self, data_dir, num_cols, cat_cols, label_col, num_workers=2,
                 batch_size_train=128, batch_size_val=64, batch_size_test=512):
        super().__init__()
        self.data_dir = data_dir
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.label_col = label_col
        self.num_workers = num_workers
        self.batch_size_train = batch_size_train
        self.batch_size_val = batch_size_val
        self.batch_size_test = batch_size_test

    def setup(self, stage):
        num_cols = self.num_cols
        cat_cols = self.cat_cols
        label_col = self.label_col
        
        path_train = os.path.join(self.data_dir, 'train.csv')
        self.dataset_train = TabularDataset(path_train, num_cols, cat_cols, label_col)
        #self.dataset_train= df_shrink(self.dataset_train)

        path_val = os.path.join(self.data_dir, 'val.csv')
        self.dataset_val = TabularDataset(path_val, num_cols, cat_cols, label_col)
        

        path_test = os.path.join(self.data_dir, 'test.csv')
        self.dataset_test = TabularDataset(path_test, num_cols, cat_cols, label_col)
        #self.dataset_test= df_shrink(self.dataset_test)

    def train_dataloader(self):
        return DataLoader(
            self.dataset_train,
            num_workers=self.num_workers,
            batch_size=self.batch_size_train,
            shuffle=True
        )

    def val_dataloader(self):
        return DataLoader(
            self.dataset_val,
            num_workers=self.num_workers,
            batch_size=self.batch_size_val,
            shuffle=False
        )

    def test_dataloader(self):
        return DataLoader(
            self.dataset_test,
            num_workers=self.num_workers,
            batch_size=self.batch_size_test,
            shuffle=False
        )

In [16]:
class TabularNet(pl.LightningModule):

    def __init__(self, num_cols, cat_cols, embedding_size_dict, n_classes,
                 embedding_dim_dict=None, learning_rate=0.01):
        super().__init__()
        
        # pytorch lightning black magic, all the arguments can now be
        # accessed through self.hparams.[argument]
        self.save_hyperparameters()

        self.embeddings, total_embedding_dim = self._create_embedding_layers(
            cat_cols, embedding_size_dict, embedding_dim_dict)
        
        # concatenate the numerical variables and the embedding layers
        # then proceed with the rest of the sequential flow
        in_features = len(num_cols) + total_embedding_dim
        self.layers = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, n_classes)
        )

    @staticmethod
    def _create_embedding_layers(cat_cols, embedding_size_dict, embedding_dim_dict):
        """construct the embedding layer, 1 per each categorical variable"""
        total_embedding_dim = 0
        embeddings = {}
        for col in cat_cols:
            embedding_size = embedding_size_dict[col]
            embedding_dim = embedding_dim_dict[col]
            total_embedding_dim += embedding_dim
            embeddings[col] = nn.Embedding(embedding_size, embedding_dim)

        return nn.ModuleDict(embeddings), total_embedding_dim

    def forward(self, num_tensor, cat_tensor):

        # run through all the categorical variables through its
        # own embedding layer and concatenate them together
        cat_outputs = []
        for i, col in enumerate(self.hparams.cat_cols):
            embedding = self.embeddings[col]
            cat_output = embedding(cat_tensor[:, i])
            cat_outputs.append(cat_output)
        
        cat_outputs = torch.cat(cat_outputs, dim=1)
        
        # concatenate the categorical embedding and numerical layer
        all_outputs = torch.cat((num_tensor, cat_outputs), dim=1)
        
        # for binary classification or regression we don't need the additional dimension
        final_outputs = self.layers(all_outputs).squeeze(dim=1)
        return final_outputs

    def training_step(self, batch, batch_idx):
        loss = self.compute_loss(batch, batch_idx)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.compute_loss(batch, batch_idx)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss = self.compute_loss(batch, batch_idx)
        self.log('test_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def compute_loss(self, batch, batch_idx):
        num_tensor, cat_tensor, label_tensor = batch
        output_tensor = self(num_tensor, cat_tensor)
        loss = F.binary_cross_entropy_with_logits(output_tensor, label_tensor)
        return loss

In [17]:
def emb_sz_rule(n_cat):
    "Rule of thumb to pick embedding size corresponding to `n_cat`"
    return min(600, round(1.6 * n_cat ** 0.56))

In [18]:
n_classes = 3

embedding_size_dict = {col: len(code) for col, code in cat_code_dict.items()}
embedding_dim_dict = {col: embedding_size // 2 for col, embedding_size in embedding_size_dict.items()}
embedding_dim_dict

{'timestampYear': 6,
 'timestampMonth': 6,
 'timestampWeek': 26,
 'timestampDay': 15,
 'timestampDayofweek': 3,
 'timestampDayofyear': 177,
 'timestampIs_month_end': 1,
 'timestampIs_month_start': 1,
 'timestampIs_quarter_end': 1,
 'timestampIs_quarter_start': 1,
 'timestampIs_year_end': 1,
 'timestampIs_year_start': 1,
 'timestampElapsed': 110953}

In [ ]:
tabular_data_module = TabularDataModule(data_dir, cols, cat_cols, target)

# we can print out the network architecture for inspection
tabular_model = TabularNet(cols, cat_cols, embedding_size_dict, n_classes, embedding_dim_dict)
tabular_model

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor='val_loss')]
trainer = pl.Trainer(max_epochs=8, callbacks=callbacks)
trainer.fit(tabular_model, tabular_data_module)

In [ ]:
tabular_model.eval()
with torch.no_grad():
    model_pred = tabular_model(num_tensor, cat_tensor).cpu().numpy()

model_pred

neutral    36686
put        14284
call       13171
Name: prediction, dtype: int64

In [ ]:
def predict(tabular_model, tabular_data_module):
    data_loader = tabular_data_module.test_dataloader()
    batch_size = data_loader.batch_size
    n_rows = len(tabular_data_module.dataset_test)
    
    y_true = np.zeros(n_rows, dtype=np.float32)
    y_pred = np.zeros(n_rows, dtype=np.float32)
    with torch.no_grad():
        idx = 0
        for num_batch, cat_batch, label_batch in data_loader:
            y_output = tabular_model(num_batch, cat_batch)

            # we convert the output value to binary classification probability
            # with a sigmoid operation, note that this step is specific to the
            # problem at hand, and might not apply to say a regression problem
            y_prob = torch.sigmoid(y_output).cpu().numpy()

            start_idx = idx
            idx += batch_size
            end_idx = idx
            y_pred[start_idx:end_idx] = y_prob
            y_true[start_idx:end_idx] = label_batch.cpu().numpy()

            if end_idx == n_rows:
                break

    return y_true, y_pred

neutral    28721
call       17744
put        17676
Name: label__minus_15_1min_close__label_generator_4class, dtype: int64

In [ ]:
y_true, y_pred = predict(tabular_model, tabular_data_module)
y_true, y_pred

   | Name                                    | Type                      | Params
---------------------------------------------------------------------------------------
0  | custom_loss                             | CrossEntropyLoss          | 0     
1  | _backbone                               | CategoryEmbeddingBackbone | 1.2 M 
2  | _backbone.linear_layers                 | Sequential                | 1.2 M 
3  | _backbone.linear_layers.0               | Linear                    | 375 K 
4  | _backbone.linear_layers.1               | LeakyReLU                 | 0     
5  | _backbone.linear_layers.2               | Linear                    | 524 K 
6  | _backbone.linear_layers.3               | LeakyReLU                 | 0     
7  | _backbone.linear_layers.4               | Linear                    | 262 K 
8  | _backbone.linear_layers.5               | LeakyReLU                 | 0     
9  | _embedding_layer                        | Embedding1dLayer          | 732   
10 | _embe

In [ ]:
import sklearn.metrics as metrics


def compute_score(y_true, y_pred, round_digits=3):
    log_loss = round(metrics.log_loss(y_true, y_pred), round_digits)
    auc = round(metrics.roc_auc_score(y_true, y_pred), round_digits)

    precision, recall, threshold = metrics.precision_recall_curve(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall)

    mask = ~np.isnan(f1)
    f1 = f1[mask]
    precision = precision[mask]
    recall = recall[mask]

    best_index = np.argmax(f1)
    threshold = round(threshold[best_index], round_digits)
    precision = round(precision[best_index], round_digits)
    recall = round(recall[best_index], round_digits)
    f1 = round(f1[best_index], round_digits)

    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'threshold': threshold,
        'log_loss': log_loss
    }

In [ ]:
compute_score(y_true, y_pred)


In [ ]:
tabular_model_loaded = TabularNet.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

tabular_model_loaded.eval()
with torch.no_grad():
    model_pred_loaded = tabular_model_loaded(num_tensor, cat_tensor).cpu().numpy()

y_true, y_pred = predict(tabular_model_loaded, tabular_data_module)
compute_score(y_true, y_pred)

In [ ]:
import sklearn.metrics as mt
c = mt.classification_report(q['label__minus_15_1min_close__label_generator_4class'],q['prediction'])

In [ ]:
print(c)

              precision    recall  f1-score   support

        call       0.35      0.26      0.29     17744
     neutral       0.56      0.72      0.63     28721
         put       0.37      0.30      0.33     17676

    accuracy                           0.47     64141
   macro avg       0.42      0.42      0.42     64141
weighted avg       0.45      0.47      0.45     64141



In [ ]:
c1 = mt.confusion_matrix(q['label__minus_15_1min_close__label_generator_4class'],q['prediction'])

In [ ]:
c1

array([[ 4553,  8356,  4835],
       [ 3946, 20544,  4231],
       [ 4672,  7786,  5218]])